Author: Adafaly Matthieu </br>

This notebook is used to prepare the Data_Exploration notebook.
Its purpose is to create a new dataset that combines a time filter and a location filter, allowing you to analyze a subset of the data according to your preferences.

# Importation of the libraries


In [1]:
import pandas as pd
from IPython.display import display, Markdown
from datetime import datetime, timedelta

# Data

In [2]:
df = pd.read_pickle("Data/pollution_rennes.pkl")
df=df.reset_index()
print("dataframe loaded")

dataframe loaded


Creating the dataframe for fixed stations and the one for mobile stations.

In [3]:
df_stationnary = df.loc[(df['sensor_type'] == 'fixedGps') & (df['PM_2.5'].notna())]
df_mobile = df.loc[(df['sensor_type'] == 'mobileGps') & (df['PM_2.5'].notna())]

In [4]:
df.groupby('sensor_name')['sensor_type'].unique()

sensor_name
parautarin02            [mobileGps]
parautarin30            [mobileGps]
parautarin31            [mobileGps]
parautarin32            [mobileGps]
parautarin33            [mobileGps]
parautarin34            [mobileGps]
parautarin35            [mobileGps]
standalone-LOPY-AQ03     [fixedGps]
standalone-LOPY-AQ09     [fixedGps]
Name: sensor_type, dtype: object

In [5]:
df_comparaison_sensor = pd.DataFrame({
    'Total': df['sensor_name'].value_counts().sort_index(),
}).fillna(0).astype(int)

display(Markdown("### 📊 Chart of the repartion if the pollution have a value or not by sensor"))
display(df_comparaison_sensor)

### 📊 Chart of the repartion if the pollution have a value or not by sensor

,Total
sensor_name,
parautarin02,111454
parautarin30,1239
parautarin31,8506
parautarin32,16111
parautarin33,116
parautarin34,12308
parautarin35,60431
standalone-LOPY-AQ03,403859
standalone-LOPY-AQ09,435155


In [6]:
# Create a series representing the month indices (1 to 12)
month_index = pd.Series(range(1, 13), name="month")
# Create a DataFrame 'df_sensor_type_comparison' by counting occurrences of 'mois' in different DataFrames
df_sensor_type_comparison = pd.DataFrame({
    'global': df['month'].value_counts().reindex(month_index).fillna(0),  # Counts from the 'df' DataFrame
    'mobile': df_mobile['month'].value_counts().reindex(month_index).fillna(0),  # Counts from 'df_mobile'
    'stationnary': df_stationnary['month'].value_counts().reindex(month_index).fillna(0)  # Counts from 'df_stationnary'
}).astype(int)  # Ensures all values are integers

month_names = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "October", 11: "November", 12: "December"
}


# Replace the index with the month names instead of month numbers
df_sensor_type_comparison.index = df_sensor_type_comparison.index.map(month_names)

# Calculate the total row by summing the columns
total_row = df_sensor_type_comparison[['global', 'mobile', 'stationnary']].sum()

# Add the total row to the DataFrame
df_sensor_type_comparison.loc['Total'] = total_row

# Display the title as markdown
display(Markdown("### 📊 Chart of the number of measure by month"))

# Display the resulting DataFrame
display(df_sensor_type_comparison)

### 📊 Chart of the number of measure by month

,global,mobile,stationnary
month,,,
January,0,0,0
February,0,0,0
March,0,0,0
April,49185,2932,46253
May,111918,9119,102799
June,111987,4893,107094
July,120966,23051,97915
August,119001,19156,99845
September,101320,17026,84294


# Analysis


In [7]:
# Convert the 'measure_date' column to datetime
df['date'] = pd.to_datetime(df['measure_date'], format='%Y-%m-%d', utc=True)

# Function to validate a date in the YYYY-MM-DD format
def validate_date(date_str):
    try:
        # Try to convert the string to a date object
        datetime.strptime(date_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False

# Ask the user for the start and end date
start_date = '2020-04-11'
end_date = "2020-12-16"

# If the user enters "start", use the first date from the DataFrame
if start_date.lower() == "start":
    start_date = df['date'].min().strftime('%Y-%m-%d')  # Use the earliest date in the DataFrame
    print(f"Start date set to: {start_date}")

# If the user enters "end", use the last date from the DataFrame
if end_date.lower() == "end":
    end_date = df['date'].max().strftime('%Y-%m-%d')  # Use the latest date in the DataFrame
    print(f"End date set to: {end_date}")

# Check that the dates are filled and in the correct format
if start_date and end_date:
    if validate_date(start_date) and validate_date(end_date):
        print("Dates are valid.")
    else:
        print("One or more dates are invalid. Please use the YYYY-MM-DD format.")
else:
    print("Date fields cannot be empty.")

# Convert start and end dates to datetime with the desired time
start_date = datetime.strptime(start_date + ' 00:00', '%Y-%m-%d %H:%M')
end_date = datetime.strptime(end_date + ' 23:59', '%Y-%m-%d %H:%M')

# Also convert them to datetime64[ns, UTC] for comparison with pandas
start_date = pd.to_datetime(start_date, utc=True)
end_date = pd.to_datetime(end_date, utc=True)

# Filter the DataFrame based on the selected date range
filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

# Display the filtered results
print("Filtered data:")
display(filtered_df)

Dates are valid.
Filtered data:


,sensor_name,measure_date,id,geo_type,geo_coords,start_date,sensor_type,p,PM_2.5,bn,longitude,latitude,day_week,day,hour,month,year,hour_minute_second,date
0,parautarin02,2020-04-14 10:44:11+00:00,5e959400c541160cddaff6ab,LineString,"[[-1.6081071260000002, 48.114313352], [-1.6075...",2020-04-14 08:43:56+00:00,mobileGps,24.045,7.28,OPC_N3:12,-1.607370,48.114360,Tuesday,14,10,4,2020,10:44:11,2020-04-14 10:44:11+00:00
1,parautarin02,2020-04-14 10:44:36+00:00,5e959418c541160cddaff6b2,LineString,"[[-1.611558352, 48.114267077], [-1.610887129, ...",2020-04-14 08:44:17+00:00,mobileGps,24.063,5.21,OPC_N3:12,-1.610273,48.114306,Tuesday,14,10,4,2020,10:44:36,2020-04-14 10:44:36+00:00
2,parautarin02,2020-04-14 10:45:02+00:00,5e959430c541160cddaff6bb,LineString,"[[-1.615186739, 48.11430112], [-1.614956631, 4...",2020-04-14 08:44:41+00:00,mobileGps,24.023,5.16,OPC_N3:12,-1.614167,48.114261,Tuesday,14,10,4,2020,10:45:02,2020-04-14 10:45:02+00:00
3,parautarin02,2020-04-14 10:45:26+00:00,5e959448c541160cddaff6c2,LineString,"[[-1.617924945, 48.11421987], [-1.617159374, 4...",2020-04-14 08:45:07+00:00,mobileGps,24.058,6.08,OPC_N3:12,-1.616512,48.114245,Tuesday,14,10,4,2020,10:45:26,2020-04-14 10:45:26+00:00
4,parautarin02,2020-04-14 10:45:47+00:00,5e959460c541160cddaff6ca,LineString,"[[-1.620713292, 48.114450095], [-1.620169099, ...",2020-04-14 08:45:32+00:00,mobileGps,24.040,5.48,OPC_N3:12,-1.619935,48.114274,Tuesday,14,10,4,2020,10:45:47,2020-04-14 10:45:47+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049174,standalone-LOPY-AQ09,2020-12-16 01:01:21+00:00,5fd95c7ce5283f00124f1692,Point,"[-1.651349, 48.120699]",2020-05-14 12:35:37+00:00,fixedGps,34.079,6.23,OPC_N3:05,-1.651349,48.120699,Wednesday,16,1,12,2020,01:01:21,2020-12-16 01:01:21+00:00
1049175,standalone-LOPY-AQ09,2020-12-16 01:01:55+00:00,5fd95c9ce5283f00124f1696,Point,"[-1.651349, 48.120699]",2020-05-14 12:35:37+00:00,fixedGps,34.080,6.83,OPC_N3:05,-1.651349,48.120699,Wednesday,16,1,12,2020,01:01:55,2020-12-16 01:01:55+00:00
1049176,standalone-LOPY-AQ09,2020-12-16 01:02:29+00:00,5fd95cbae5283f00124f169a,Point,"[-1.651349, 48.120699]",2020-05-14 12:35:37+00:00,fixedGps,34.063,7.50,OPC_N3:05,-1.651349,48.120699,Wednesday,16,1,12,2020,01:02:29,2020-12-16 01:02:29+00:00
1049177,standalone-LOPY-AQ09,2020-12-16 01:03:03+00:00,5fd95ce0e5283f00124f169c,Point,"[-1.651349, 48.120699]",2020-05-14 12:35:37+00:00,fixedGps,34.060,7.87,OPC_N3:05,-1.651349,48.120699,Wednesday,16,1,12,2020,01:03:03,2020-12-16 01:03:03+00:00


In [8]:
# List of sensor available
print(f"sensor available {df['sensor_name'].unique()}")
sensor_selected = ['standalone-LOPY-AQ03', 'parautarin02', 'standalone-LOPY-AQ09', 'parautarin31', 'parautarin30', 'standalone-LOPY-AQ05', 'parautarin34', 'parautarin33', 'parautarin32', 'parautarin35']
print(f'sensor selected({sensor_selected})')

sensor available ['parautarin02' 'parautarin30' 'parautarin31' 'parautarin32'
 'parautarin33' 'parautarin34' 'parautarin35' 'standalone-LOPY-AQ03'
 'standalone-LOPY-AQ09']
sensor selected(['standalone-LOPY-AQ03', 'parautarin02', 'standalone-LOPY-AQ09', 'parautarin31', 'parautarin30', 'standalone-LOPY-AQ05', 'parautarin34', 'parautarin33', 'parautarin32', 'parautarin35'])


In [9]:
print(sensor_selected)
print(start_date)
print(end_date)

['standalone-LOPY-AQ03', 'parautarin02', 'standalone-LOPY-AQ09', 'parautarin31', 'parautarin30', 'standalone-LOPY-AQ05', 'parautarin34', 'parautarin33', 'parautarin32', 'parautarin35']
2020-04-11 00:00:00+00:00
2020-12-16 23:59:00+00:00


In [10]:
df_filter = df[(df['measure_date'] >= start_date) & (df['measure_date'] <= end_date) & (df['sensor_name'].isin(sensor_selected))]
print("Data filter :")
display(df_filter)

Data filter :


,sensor_name,measure_date,id,geo_type,geo_coords,start_date,sensor_type,p,PM_2.5,bn,longitude,latitude,day_week,day,hour,month,year,hour_minute_second,date
0,parautarin02,2020-04-14 10:44:11+00:00,5e959400c541160cddaff6ab,LineString,"[[-1.6081071260000002, 48.114313352], [-1.6075...",2020-04-14 08:43:56+00:00,mobileGps,24.045,7.28,OPC_N3:12,-1.607370,48.114360,Tuesday,14,10,4,2020,10:44:11,2020-04-14 10:44:11+00:00
1,parautarin02,2020-04-14 10:44:36+00:00,5e959418c541160cddaff6b2,LineString,"[[-1.611558352, 48.114267077], [-1.610887129, ...",2020-04-14 08:44:17+00:00,mobileGps,24.063,5.21,OPC_N3:12,-1.610273,48.114306,Tuesday,14,10,4,2020,10:44:36,2020-04-14 10:44:36+00:00
2,parautarin02,2020-04-14 10:45:02+00:00,5e959430c541160cddaff6bb,LineString,"[[-1.615186739, 48.11430112], [-1.614956631, 4...",2020-04-14 08:44:41+00:00,mobileGps,24.023,5.16,OPC_N3:12,-1.614167,48.114261,Tuesday,14,10,4,2020,10:45:02,2020-04-14 10:45:02+00:00
3,parautarin02,2020-04-14 10:45:26+00:00,5e959448c541160cddaff6c2,LineString,"[[-1.617924945, 48.11421987], [-1.617159374, 4...",2020-04-14 08:45:07+00:00,mobileGps,24.058,6.08,OPC_N3:12,-1.616512,48.114245,Tuesday,14,10,4,2020,10:45:26,2020-04-14 10:45:26+00:00
4,parautarin02,2020-04-14 10:45:47+00:00,5e959460c541160cddaff6ca,LineString,"[[-1.620713292, 48.114450095], [-1.620169099, ...",2020-04-14 08:45:32+00:00,mobileGps,24.040,5.48,OPC_N3:12,-1.619935,48.114274,Tuesday,14,10,4,2020,10:45:47,2020-04-14 10:45:47+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049174,standalone-LOPY-AQ09,2020-12-16 01:01:21+00:00,5fd95c7ce5283f00124f1692,Point,"[-1.651349, 48.120699]",2020-05-14 12:35:37+00:00,fixedGps,34.079,6.23,OPC_N3:05,-1.651349,48.120699,Wednesday,16,1,12,2020,01:01:21,2020-12-16 01:01:21+00:00
1049175,standalone-LOPY-AQ09,2020-12-16 01:01:55+00:00,5fd95c9ce5283f00124f1696,Point,"[-1.651349, 48.120699]",2020-05-14 12:35:37+00:00,fixedGps,34.080,6.83,OPC_N3:05,-1.651349,48.120699,Wednesday,16,1,12,2020,01:01:55,2020-12-16 01:01:55+00:00
1049176,standalone-LOPY-AQ09,2020-12-16 01:02:29+00:00,5fd95cbae5283f00124f169a,Point,"[-1.651349, 48.120699]",2020-05-14 12:35:37+00:00,fixedGps,34.063,7.50,OPC_N3:05,-1.651349,48.120699,Wednesday,16,1,12,2020,01:02:29,2020-12-16 01:02:29+00:00
1049177,standalone-LOPY-AQ09,2020-12-16 01:03:03+00:00,5fd95ce0e5283f00124f169c,Point,"[-1.651349, 48.120699]",2020-05-14 12:35:37+00:00,fixedGps,34.060,7.87,OPC_N3:05,-1.651349,48.120699,Wednesday,16,1,12,2020,01:03:03,2020-12-16 01:03:03+00:00


In [11]:
df_filter = df_filter.set_index(["sensor_name", "measure_date"])
df_filter.to_pickle('Data/pollution_rennes_filter.pkl')